In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf


In [2]:
tf.test.is_gpu_available()

True

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import tensorflow.keras.applications.resnet50 as resnet50 
import tensorflow.keras.applications.vgg19 as vgg19
import tensorflow.keras.applications.densenet as densenet
import tensorflow.keras.applications.inception_v3 as inception_v3

In [5]:
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.losses as losses
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.backend as K

# Dataset

In [6]:
from scipy.io import loadmat
from os.path import join

prefix = './Stanford_Dog_Breed'
imgfix = 'Images'
anofix = 'Annotation'

In [7]:
ImgTrainGen = image.ImageDataGenerator(preprocessing_function=densenet.preprocess_input, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  shear_range=0.2, 
                                  zoom_range=0.2,
#                                   rotation_range=0.2,
                                  fill_mode='nearest',
#                                   horizontal_flip=True, 
#                                   validation_split=1/6.
                                      )

Train = ImgTrainGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
                                   target_size=(224, 224), 
                                   class_mode='sparse', 
                                   batch_size=32, 
                                   shuffle=True, 
                                   seed=None, 
                                   subset='training', 
                                   interpolation='nearest')

ImgTestGen = image.ImageDataGenerator(preprocessing_function=densenet.preprocess_input, validation_split=1/6.)

Test  = ImgTestGen.flow_from_directory(join(prefix, imgfix, 'Test' ), 
                                       target_size=(224, 224), 
                                       class_mode='sparse', 
                                       batch_size=32, 
                                       shuffle=False, 
                                       interpolation='nearest')

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [8]:
def MultiTaskDataGen(ImgGen):
    while True:
        X = ImgGen.next()
        yield [X[0], X[1]], [X[1], X[1], X[1]]
        
TrainData = MultiTaskDataGen(Train)
# ValidData = MultiTaskDataGen(Valid)
TestData  = MultiTaskDataGen(Test)

# Densenet 121

In [9]:
densenet121_naive = densenet.DenseNet121(weights='imagenet', include_top=False)

In [10]:
input_image = densenet121_naive.input

In [11]:
x = densenet121_naive.output
print x.shape

# x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
# print x.shape

x = layers.GlobalAveragePooling2D()(x)
print x.shape
x = layers.Dropout(0.2)(x)
x = layers.Dense(120, activation='softmax')(x)
print(x.shape)
# # x = layers.Dense(1024, activation='relu')(x)
# # print x.shape
# pred = layers.Dense(120, activation='softmax')(x)
# print pred.shape

(?, ?, ?, 1024)
(?, 1024)
(?, 120)


In [12]:
model = models.Model(inputs=input_image, outputs=x)
for layer in densenet121_naive.layers:
    layer.trainable = False

model.compile(
    optimizer='SGD', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])



In [13]:
# Callbacks
class SaveModelandEval(callbacks.Callback):
    prev_res = list()
    def __init__(self):
        self.prev_res = [0., 0.]
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 10 == 0:
            res = model.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)
            print('\n', res)
            if res[1] > self.prev_res[1]:
                models.save_model(model, 'densenet121_dfc120_%s_%s'%(epoch, int(res[1]*10000)) )
                self.prev_res = res

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [10]:
model = models.load_model('densenet121_dfc120_120_8059')

In [14]:
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD

In [15]:
model.compile(
#     optimizer=SGD(lr=0.0001, momentum=0.9),
    optimizer='SGD',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

In [14]:
history_densenet121=model.fit_generator(
    Train, 
    steps_per_epoch=80, 
    epochs=122, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=0,
    callbacks=[SaveModelandEval()]
)

Epoch 1/122
80/80 [==============================] - 15s 181ms/step - loss: 5.0546 - acc: 0.0148
Epoch 2/122
80/80 [==============================] - 11s 137ms/step - loss: 4.5068 - acc: 0.0523
Epoch 3/122
80/80 [==============================] - 11s 135ms/step - loss: 4.0802 - acc: 0.1020
Epoch 4/122
80/80 [==============================] - 11s 137ms/step - loss: 3.7594 - acc: 0.1633
Epoch 5/122
80/80 [==============================] - 14s 178ms/step - loss: 3.3724 - acc: 0.2434
Epoch 6/122
80/80 [==============================] - 19s 242ms/step - loss: 3.0790 - acc: 0.3227
Epoch 7/122
80/80 [==============================] - 19s 234ms/step - loss: 2.7982 - acc: 0.3848
Epoch 8/122
80/80 [==============================] - 14s 179ms/step - loss: 2.6645 - acc: 0.3855
Epoch 9/122
80/80 [==============================] - 13s 167ms/step - loss: 2.4905 - acc: 0.4418
Epoch 10/122
80/80 [==============================] - 18s 226ms/step - loss: 2.3211 - acc: 0.4801
Epoch 11/122
80/80 [=========

In [ ]:
models.save_model(densenet121_modified, 'densenet_60_finetune422_5')

In [15]:
import json

with open('history_densenet121.json', 'w') as f:
    json.dump(history_densenet121.history, f)

# Fine Tune

In [44]:
model = models.load_model('densenet121_dfc120_finetune_9_8141')

In [18]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

(0, u'input_2')
(1, u'zero_padding2d_2')
(2, u'conv1/conv')
(3, u'conv1/bn')
(4, u'conv1/relu')
(5, u'zero_padding2d_3')
(6, u'pool1')
(7, u'conv2_block1_0_bn')
(8, u'conv2_block1_0_relu')
(9, u'conv2_block1_1_conv')
(10, u'conv2_block1_1_bn')
(11, u'conv2_block1_1_relu')
(12, u'conv2_block1_2_conv')
(13, u'conv2_block1_concat')
(14, u'conv2_block2_0_bn')
(15, u'conv2_block2_0_relu')
(16, u'conv2_block2_1_conv')
(17, u'conv2_block2_1_bn')
(18, u'conv2_block2_1_relu')
(19, u'conv2_block2_2_conv')
(20, u'conv2_block2_concat')
(21, u'conv2_block3_0_bn')
(22, u'conv2_block3_0_relu')
(23, u'conv2_block3_1_conv')
(24, u'conv2_block3_1_bn')
(25, u'conv2_block3_1_relu')
(26, u'conv2_block3_2_conv')
(27, u'conv2_block3_concat')
(28, u'conv2_block4_0_bn')
(29, u'conv2_block4_0_relu')
(30, u'conv2_block4_1_conv')
(31, u'conv2_block4_1_bn')
(32, u'conv2_block4_1_relu')
(33, u'conv2_block4_2_conv')
(34, u'conv2_block4_concat')
(35, u'conv2_block5_0_bn')
(36, u'conv2_block5_0_relu')
(37, u'conv2_blo

In [45]:
for layer in model.layers[:385]:
    layer.trainable = False
for layer in model.layers[385:]:
    layer.trainable = True

In [40]:
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD

In [46]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [36]:
# Callbacks
class SaveModelandEval_finetune(callbacks.Callback):
    prev_res = list()
    def __init__(self):
        self.prev_res = [0., 0.]
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 3 == 0:
            res = model.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)
            print('\n', res)
            if res[1] > self.prev_res[1]:
                models.save_model(model, 'densenet121_dfc120_finetune_%s_%s'%(epoch, int(res[1]*10000)) )
                self.prev_res = res

In [47]:
model.fit_generator(
    Train, 
    steps_per_epoch=80, 
    epochs=22, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=0,
    callbacks=[SaveModelandEval_finetune()]
)

Epoch 1/22
80/80 [==============================] - 41s 510ms/step - loss: 0.7602 - acc: 0.7852
Epoch 2/22
80/80 [==============================] - 15s 190ms/step - loss: 0.7771 - acc: 0.7742
Epoch 3/22
80/80 [==============================] - 14s 180ms/step - loss: 0.7410 - acc: 0.7875
Epoch 4/22
80/80 [==============================] - 332s 4s/step - loss: 0.7593 - acc: 0.7770
Epoch 5/22
80/80 [==============================] - 14s 173ms/step - loss: 0.7558 - acc: 0.7828
Epoch 6/22
80/80 [==============================] - 17s 214ms/step - loss: 0.7336 - acc: 0.7938
Epoch 7/22
80/80 [==============================] - 51s 632ms/step - loss: 0.7653 - acc: 0.7848
Epoch 8/22
80/80 [==============================] - 12s 148ms/step - loss: 0.7288 - acc: 0.7965
Epoch 9/22
80/80 [==============================] - 14s 180ms/step - loss: 0.7590 - acc: 0.7770
Epoch 10/22
80/80 [==============================] - 58s 729ms/step - loss: 0.7681 - acc: 0.7805
Epoch 11/22
80/80 [======================